# Assignment2
## Anomaly detection 데이터셋을 SVM을 이용해서 판별해보자
### 데이터 분포 : True : 0.17프로 나머지 다 False, Row약 28만행 < 데이터 트레인 시간이 오래걸립니다.(그래도 자고 일어나면 트레이닝 돼있기는 하는정도입니다.!!)
### 여러가지 방법을 사용해서 트레이닝 해보고 결과를 비교해주면됩니다!
### 시간이 없거나 좀 어렵다 하시는 분들은 멘토에게 물어보고 한 가지 방법으로만 구현해도 됩니다.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [2]:
# Anomaly detection(사기감지 데이터) 로드
data = pd.read_csv('../anomaly-detection/creditcard.csv')
print(data.columns)

Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


In [3]:
data.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0


In [4]:
y = data['Class']    # 타겟분리
data2 = data.drop('Class',axis=1)
X = data2
X = data2.values
data2.head(3)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66


## 진행 순서
1. 데이터 불러오기 및 타겟 변수 분리
2. 데이터셋 분리
3. imbalance 데이터 처리
4. 데이터 스케일링

imbalance 데이터 처리 https://joonable.tistory.com/27  

SMOTE를 통한 데이터 불균형 처리 https://mkjjo.github.io/python/2019/01/04/smote_duplicate.html  

RandomUnderSampling 설명  
http://glemaitre.github.io/imbalanced-learn/generated/imblearn.under_sampling.RandomUnderSampler.html

In [5]:
# 데이터셋 분리
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=0)

혹시 데이터셋 분리 전에 데이터 불균형 처리를 먼저하는 것이 좋을까요? 순서가 헷갈립니다!

In [6]:
# imbalanced 데이터 처리 위해 imbalanced-learn패키지 사용
# 데이터 양이 많으므로 짧은 시간내에 정확도 높이고자 언더 샘플링함

from imblearn.under_sampling import *

In [7]:
# 데이터 언더샘플링
X_under,y_under = RandomUnderSampler(random_state=0).fit_sample(X_train,y_train)

In [8]:
# 데이터 스케일링
ss = StandardScaler()

In [9]:
X_u_s = ss.fit_transform(X_under)

In [10]:
from sklearn.metrics.pairwise import rbf_kernel 

# linear한 SVM적용, 에러패널티 C는 100으로 높음
svc = SVC(kernel='linear',C=100)

# 모델 트레이닝
svc.fit(X_u_s,y_under)

SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [11]:
# test 및 성능평가
y_pred=svc.predict(X_test)
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score:
0.9946279976124434


In [12]:
# 적절한 C값을 찾아보자
from tqdm import tqdm 
from sklearn.model_selection import cross_validate

In [24]:
# kernel rbf 사용

C_range = list(np.arange(1,100,1))
acc_score = []

# cross validation
for c in tqdm(C_range):
    scv = SVC(kernel = 'rbf',C=c,gamma = 'scale')
    scores = cross_validate(svc,X_u_s,y_under,cv=10,scoring='accuracy')
    score = np.mean(scores['test_score'])
    acc_score.append(score)
print(np.max(acc_score))
print(C_range[np.argmax(acc_score)])



100%|██████████████████████████████████████████████████████████████████████████████████| 99/99 [06:14<00:00,  3.79s/it]

0.9358974358974359
1


In [27]:
# 가장 성능이 좋다고 나온 C=1으로 SCV 진행, kernel은 rbf, gamma는 디폴트 값 사용
svc_c = SVC(kernel='rbf',C=1,gamma = 'scale')

# 모델 트레이닝
svc_c.fit(X_u_s,y_under)

# # test 및 성능평가
y_pred=svc_c.predict(X_test)
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score:
0.0017906674625188723


In [28]:
# rbf 사용시 낮은 성능을 보여서 linear사용하고 다시 C값을 찾아봄

C_range = list(np.arange(1,100,1))
acc_score = []

# cross validation
for c in tqdm(C_range):
    scv = SVC(kernel = 'linear',C=c,gamma = 'scale')
    scores = cross_validate(svc,X_u_s,y_under,cv=10,scoring='accuracy')
    score = np.mean(scores['test_score'])
    acc_score.append(score)
print(np.max(acc_score))
print(C_range[np.argmax(acc_score)])

100%|██████████████████████████████████████████████████████████████████████████████████| 99/99 [06:13<00:00,  3.77s/it]

0.9358974358974359
1


### 0.9358974358974359,1  
linear와 rbf 커널 모두 값이 동일하게 나옴 왜그런지 모르겠습니다..  
커널은 상관없나요??

In [29]:
# 가장 성능이 좋다고 나온 C=1으로 SCV 진행, kernel은 rbf, gamma는 디폴트 값 사용
svc_c = SVC(kernel='linear',C=1,gamma = 'scale')

# 모델 트레이닝
svc_c.fit(X_u_s,y_under)

# # test 및 성능평가
y_pred=svc_c.predict(X_test)
print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score:
0.9981039991573329


In [ ]:
# cross_validate 할 때 X,y값 무엇을 넣어줘야하는지 의문!

In [14]:
# Grid Search로 파라미터 찾아보기
tuned_parameters = {
 'C': (np.arange(1,10,1)), 'kernel': ['linear'],
 'C': (np.arange(1,10,1)), 'gamma': (np.arange(1,5.1)), 'kernel': ['rbf']
                   }
from sklearn.model_selection import GridSearchCV
svm_model= SVC()
model_svm = GridSearchCV(svm_model, tuned_parameters,cv=10,scoring='accuracy', verbose = 3)

In [16]:
model_svm.fit(X_u_s,y_under)
print(model_svm.best_score_)

Fitting 10 folds for each of 45 candidates, totalling 450 fits
[CV] C=1, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=1, gamma=1.0, kernel=rbf, score=0.731, total=   0.0s
[CV] C=1, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=1, gamma=1.0, kernel=rbf, score=0.821, total=   0.0s
[CV] C=1, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=1, gamma=1.0, kernel=rbf, score=0.808, total=   0.0s
[CV] C=1, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=1, gamma=1.0, kernel=rbf, score=0.846, total=   0.0s
[CV] C=1, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=1, gamma=1.0, kernel=rbf, score=0.833, total=   0.0s
[CV] C=1, gamma=1.0, kernel=rbf ......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] .......... C=1, gamma=1.0, kernel=rbf, score=0.833, total=   0.0s
[CV] C=1, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=1, gamma=1.0, kernel=rbf, score=0.949, total=   0.0s
[CV] C=1, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=1, gamma=1.0, kernel=rbf, score=0.808, total=   0.0s
[CV] C=1, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=1, gamma=1.0, kernel=rbf, score=0.808, total=   0.0s
[CV] C=1, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=1, gamma=1.0, kernel=rbf, score=0.859, total=   0.0s
[CV] C=1, gamma=2.0, kernel=rbf ......................................
[CV] .......... C=1, gamma=2.0, kernel=rbf, score=0.628, total=   0.0s
[CV] C=1, gamma=2.0, kernel=rbf ......................................
[CV] .......... C=1, gamma=2.0, kernel=rbf, score=0.615, total=   0.0s
[CV] C=1, gamma=2.0, kernel=rbf ......................................
[CV] .

[CV] .......... C=2, gamma=2.0, kernel=rbf, score=0.679, total=   0.0s
[CV] C=2, gamma=2.0, kernel=rbf ......................................
[CV] .......... C=2, gamma=2.0, kernel=rbf, score=0.551, total=   0.0s
[CV] C=2, gamma=2.0, kernel=rbf ......................................
[CV] .......... C=2, gamma=2.0, kernel=rbf, score=0.615, total=   0.0s
[CV] C=2, gamma=3.0, kernel=rbf ......................................
[CV] .......... C=2, gamma=3.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=2, gamma=3.0, kernel=rbf ......................................
[CV] .......... C=2, gamma=3.0, kernel=rbf, score=0.603, total=   0.0s
[CV] C=2, gamma=3.0, kernel=rbf ......................................
[CV] .......... C=2, gamma=3.0, kernel=rbf, score=0.564, total=   0.0s
[CV] C=2, gamma=3.0, kernel=rbf ......................................
[CV] .......... C=2, gamma=3.0, kernel=rbf, score=0.564, total=   0.0s
[CV] C=2, gamma=3.0, kernel=rbf ......................................
[CV] .

[CV] .......... C=3, gamma=3.0, kernel=rbf, score=0.628, total=   0.0s
[CV] C=3, gamma=3.0, kernel=rbf ......................................
[CV] .......... C=3, gamma=3.0, kernel=rbf, score=0.538, total=   0.0s
[CV] C=3, gamma=3.0, kernel=rbf ......................................
[CV] .......... C=3, gamma=3.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=3, gamma=4.0, kernel=rbf ......................................
[CV] .......... C=3, gamma=4.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=3, gamma=4.0, kernel=rbf ......................................
[CV] .......... C=3, gamma=4.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=3, gamma=4.0, kernel=rbf ......................................
[CV] .......... C=3, gamma=4.0, kernel=rbf, score=0.564, total=   0.0s
[CV] C=3, gamma=4.0, kernel=rbf ......................................
[CV] .......... C=3, gamma=4.0, kernel=rbf, score=0.564, total=   0.0s
[CV] C=3, gamma=4.0, kernel=rbf ......................................
[CV] .

...

[CV] .......... C=6, gamma=1.0, kernel=rbf, score=0.731, total=   0.0s
[CV] C=6, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=6, gamma=1.0, kernel=rbf, score=0.821, total=   0.0s
[CV] C=6, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=6, gamma=1.0, kernel=rbf, score=0.808, total=   0.0s
[CV] C=6, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=6, gamma=1.0, kernel=rbf, score=0.846, total=   0.0s
[CV] C=6, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=6, gamma=1.0, kernel=rbf, score=0.833, total=   0.0s
[CV] C=6, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=6, gamma=1.0, kernel=rbf, score=0.833, total=   0.0s
[CV] C=6, gamma=1.0, kernel=rbf ......................................
[CV] .......... C=6, gamma=1.0, kernel=rbf, score=0.936, total=   0.0s
[CV] C=6, gamma=1.0, kernel=rbf ......................................
[CV] .

[CV] .......... C=7, gamma=2.0, kernel=rbf, score=0.603, total=   0.0s
[CV] C=7, gamma=2.0, kernel=rbf ......................................
[CV] .......... C=7, gamma=2.0, kernel=rbf, score=0.603, total=   0.0s
[CV] C=7, gamma=2.0, kernel=rbf ......................................
[CV] .......... C=7, gamma=2.0, kernel=rbf, score=0.692, total=   0.0s
[CV] C=7, gamma=2.0, kernel=rbf ......................................
[CV] .......... C=7, gamma=2.0, kernel=rbf, score=0.641, total=   0.0s
[CV] C=7, gamma=2.0, kernel=rbf ......................................
[CV] .......... C=7, gamma=2.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=7, gamma=2.0, kernel=rbf ......................................
[CV] .......... C=7, gamma=2.0, kernel=rbf, score=0.679, total=   0.0s
[CV] C=7, gamma=2.0, kernel=rbf ......................................
[CV] .......... C=7, gamma=2.0, kernel=rbf, score=0.551, total=   0.0s
[CV] C=7, gamma=2.0, kernel=rbf ......................................
[CV] .

[CV] .......... C=8, gamma=3.0, kernel=rbf, score=0.641, total=   0.0s
[CV] C=8, gamma=3.0, kernel=rbf ......................................
[CV] .......... C=8, gamma=3.0, kernel=rbf, score=0.603, total=   0.0s
[CV] C=8, gamma=3.0, kernel=rbf ......................................
[CV] .......... C=8, gamma=3.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=8, gamma=3.0, kernel=rbf ......................................
[CV] .......... C=8, gamma=3.0, kernel=rbf, score=0.628, total=   0.0s
[CV] C=8, gamma=3.0, kernel=rbf ......................................
[CV] .......... C=8, gamma=3.0, kernel=rbf, score=0.538, total=   0.0s
[CV] C=8, gamma=3.0, kernel=rbf ......................................
[CV] .......... C=8, gamma=3.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=8, gamma=4.0, kernel=rbf ......................................
[CV] .......... C=8, gamma=4.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=8, gamma=4.0, kernel=rbf ......................................
[CV] .

[CV] .......... C=9, gamma=4.0, kernel=rbf, score=0.641, total=   0.0s
[CV] C=9, gamma=4.0, kernel=rbf ......................................
[CV] .......... C=9, gamma=4.0, kernel=rbf, score=0.603, total=   0.0s
[CV] C=9, gamma=4.0, kernel=rbf ......................................
[CV] .......... C=9, gamma=4.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=9, gamma=4.0, kernel=rbf ......................................
[CV] .......... C=9, gamma=4.0, kernel=rbf, score=0.628, total=   0.0s
[CV] C=9, gamma=4.0, kernel=rbf ......................................
[CV] .......... C=9, gamma=4.0, kernel=rbf, score=0.513, total=   0.0s
[CV] C=9, gamma=4.0, kernel=rbf ......................................
[CV] .......... C=9, gamma=4.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=9, gamma=5.0, kernel=rbf ......................................
[CV] .......... C=9, gamma=5.0, kernel=rbf, score=0.577, total=   0.0s
[CV] C=9, gamma=5.0, kernel=rbf ......................................
[CV] .

[Parallel(n_jobs=1)]: Done 450 out of 450 | elapsed:   17.6s finished


In [17]:
# 최고의 결과가 나온 경우의 파라미터 값
print(model_svm.best_params_)

{'C': 1, 'gamma': 1.0, 'kernel': 'rbf'}


In [18]:
# 이를 적용해서 찍은 accuracy

svm_temp = SVC(kernel='rbf',gamma = 1.0, C = 1.0)
svm_temp.fit(X_u_s,y_under)
y_pred = svm_temp.predict(X_test)

print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score:
0.0017906674625188723


값이 왜이렇게 낮게나올까요? 그리드서치 할 때 c의 범위를 조정해보겠습니다.

In [19]:
# Grid Search로 파라미터 찾아보기2
tuned_parameters = {
 'C': (np.arange(3,5,0.2)), 'kernel': ['linear'],
 'C': (np.arange(3,5,0.2)), 'gamma': (np.arange(1,5.1)), 'kernel': ['rbf']
                   }
from sklearn.model_selection import GridSearchCV
svm_model= SVC()
model_svm = GridSearchCV(svm_model, tuned_parameters,cv=10,scoring='accuracy', verbose = 3)

In [20]:
model_svm.fit(X_u_s,y_under)
print(model_svm.best_score_)

Fitting 10 folds for each of 50 candidates, totalling 500 fits
[CV] C=3.0, gamma=1.0, kernel=rbf ....................................
[CV] ........ C=3.0, gamma=1.0, kernel=rbf, score=0.731, total=   0.0s
[CV] C=3.0, gamma=1.0, kernel=rbf ....................................
[CV] ........ C=3.0, gamma=1.0, kernel=rbf, score=0.821, total=   0.0s
[CV] C=3.0, gamma=1.0, kernel=rbf ....................................
[CV] ........ C=3.0, gamma=1.0, kernel=rbf, score=0.808, total=   0.0s
[CV] C=3.0, gamma=1.0, kernel=rbf ....................................
[CV] ........ C=3.0, gamma=1.0, kernel=rbf, score=0.846, total=   0.0s
[CV] C=3.0, gamma=1.0, kernel=rbf ....................................
[CV] ........ C=3.0, gamma=1.0, kernel=rbf, score=0.833, total=   0.0s
[CV] C=3.0, gamma=1.0, kernel=rbf ....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV] ........ C=3.0, gamma=1.0, kernel=rbf, score=0.833, total=   0.0s
[CV] C=3.0, gamma=1.0, kernel=rbf ....................................
[CV] ........ C=3.0, gamma=1.0, kernel=rbf, score=0.936, total=   0.0s
[CV] C=3.0, gamma=1.0, kernel=rbf ....................................
[CV] ........ C=3.0, gamma=1.0, kernel=rbf, score=0.808, total=   0.0s
[CV] C=3.0, gamma=1.0, kernel=rbf ....................................
[CV] ........ C=3.0, gamma=1.0, kernel=rbf, score=0.808, total=   0.0s
[CV] C=3.0, gamma=1.0, kernel=rbf ....................................
[CV] ........ C=3.0, gamma=1.0, kernel=rbf, score=0.859, total=   0.0s
[CV] C=3.0, gamma=2.0, kernel=rbf ....................................
[CV] ........ C=3.0, gamma=2.0, kernel=rbf, score=0.641, total=   0.0s
[CV] C=3.0, gamma=2.0, kernel=rbf ....................................
[CV] ........ C=3.0, gamma=2.0, kernel=rbf, score=0.628, total=   0.0s
[CV] C=3.0, gamma=2.0, kernel=rbf ....................................
[CV] .

[CV] ........ C=3.2, gamma=2.0, kernel=rbf, score=0.603, total=   0.0s
[CV] C=3.2, gamma=2.0, kernel=rbf ....................................
[CV] ........ C=3.2, gamma=2.0, kernel=rbf, score=0.692, total=   0.0s
[CV] C=3.2, gamma=2.0, kernel=rbf ....................................
[CV] ........ C=3.2, gamma=2.0, kernel=rbf, score=0.641, total=   0.0s
[CV] C=3.2, gamma=2.0, kernel=rbf ....................................
[CV] ........ C=3.2, gamma=2.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=3.2, gamma=2.0, kernel=rbf ....................................
[CV] ........ C=3.2, gamma=2.0, kernel=rbf, score=0.679, total=   0.0s
[CV] C=3.2, gamma=2.0, kernel=rbf ....................................
[CV] ........ C=3.2, gamma=2.0, kernel=rbf, score=0.551, total=   0.0s
[CV] C=3.2, gamma=2.0, kernel=rbf ....................................
[CV] ........ C=3.2, gamma=2.0, kernel=rbf, score=0.615, total=   0.0s
[CV] C=3.2, gamma=3.0, kernel=rbf ....................................
[CV] .

[CV]  C=3.4000000000000004, gamma=3.0, kernel=rbf, score=0.564, total=   0.0s
[CV] C=3.4000000000000004, gamma=3.0, kernel=rbf .....................
[CV]  C=3.4000000000000004, gamma=3.0, kernel=rbf, score=0.564, total=   0.0s
[CV] C=3.4000000000000004, gamma=3.0, kernel=rbf .....................
[CV]  C=3.4000000000000004, gamma=3.0, kernel=rbf, score=0.641, total=   0.0s
[CV] C=3.4000000000000004, gamma=3.0, kernel=rbf .....................
[CV]  C=3.4000000000000004, gamma=3.0, kernel=rbf, score=0.603, total=   0.0s
[CV] C=3.4000000000000004, gamma=3.0, kernel=rbf .....................
[CV]  C=3.4000000000000004, gamma=3.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=3.4000000000000004, gamma=3.0, kernel=rbf .....................
[CV]  C=3.4000000000000004, gamma=3.0, kernel=rbf, score=0.628, total=   0.0s
[CV] C=3.4000000000000004, gamma=3.0, kernel=rbf .....................
[CV]  C=3.4000000000000004, gamma=3.0, kernel=rbf, score=0.538, total=   0.0s
[CV] C=3.4000000000000004, g

...

[CV]  C=4.200000000000001, gamma=1.0, kernel=rbf, score=0.731, total=   0.0s
[CV] C=4.200000000000001, gamma=1.0, kernel=rbf ......................
[CV]  C=4.200000000000001, gamma=1.0, kernel=rbf, score=0.821, total=   0.0s
[CV] C=4.200000000000001, gamma=1.0, kernel=rbf ......................
[CV]  C=4.200000000000001, gamma=1.0, kernel=rbf, score=0.808, total=   0.0s
[CV] C=4.200000000000001, gamma=1.0, kernel=rbf ......................
[CV]  C=4.200000000000001, gamma=1.0, kernel=rbf, score=0.846, total=   0.0s
[CV] C=4.200000000000001, gamma=1.0, kernel=rbf ......................
[CV]  C=4.200000000000001, gamma=1.0, kernel=rbf, score=0.833, total=   0.0s
[CV] C=4.200000000000001, gamma=1.0, kernel=rbf ......................
[CV]  C=4.200000000000001, gamma=1.0, kernel=rbf, score=0.833, total=   0.0s
[CV] C=4.200000000000001, gamma=1.0, kernel=rbf ......................
[CV]  C=4.200000000000001, gamma=1.0, kernel=rbf, score=0.936, total=   0.0s
[CV] C=4.200000000000001, gamma=1.0

[CV]  C=4.400000000000001, gamma=1.0, kernel=rbf, score=0.808, total=   0.0s
[CV] C=4.400000000000001, gamma=1.0, kernel=rbf ......................
[CV]  C=4.400000000000001, gamma=1.0, kernel=rbf, score=0.859, total=   0.0s
[CV] C=4.400000000000001, gamma=2.0, kernel=rbf ......................
[CV]  C=4.400000000000001, gamma=2.0, kernel=rbf, score=0.641, total=   0.0s
[CV] C=4.400000000000001, gamma=2.0, kernel=rbf ......................
[CV]  C=4.400000000000001, gamma=2.0, kernel=rbf, score=0.628, total=   0.0s
[CV] C=4.400000000000001, gamma=2.0, kernel=rbf ......................
[CV]  C=4.400000000000001, gamma=2.0, kernel=rbf, score=0.603, total=   0.0s
[CV] C=4.400000000000001, gamma=2.0, kernel=rbf ......................
[CV]  C=4.400000000000001, gamma=2.0, kernel=rbf, score=0.603, total=   0.0s
[CV] C=4.400000000000001, gamma=2.0, kernel=rbf ......................
[CV]  C=4.400000000000001, gamma=2.0, kernel=rbf, score=0.692, total=   0.0s
[CV] C=4.400000000000001, gamma=2.0

[CV]  C=4.600000000000001, gamma=2.0, kernel=rbf, score=0.551, total=   0.0s
[CV] C=4.600000000000001, gamma=2.0, kernel=rbf ......................
[CV]  C=4.600000000000001, gamma=2.0, kernel=rbf, score=0.615, total=   0.0s
[CV] C=4.600000000000001, gamma=3.0, kernel=rbf ......................
[CV]  C=4.600000000000001, gamma=3.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=4.600000000000001, gamma=3.0, kernel=rbf ......................
[CV]  C=4.600000000000001, gamma=3.0, kernel=rbf, score=0.603, total=   0.0s
[CV] C=4.600000000000001, gamma=3.0, kernel=rbf ......................
[CV]  C=4.600000000000001, gamma=3.0, kernel=rbf, score=0.564, total=   0.0s
[CV] C=4.600000000000001, gamma=3.0, kernel=rbf ......................
[CV]  C=4.600000000000001, gamma=3.0, kernel=rbf, score=0.564, total=   0.0s
[CV] C=4.600000000000001, gamma=3.0, kernel=rbf ......................
[CV]  C=4.600000000000001, gamma=3.0, kernel=rbf, score=0.641, total=   0.0s
[CV] C=4.600000000000001, gamma=3.0

[CV] C=4.800000000000002, gamma=3.0, kernel=rbf ......................
[CV]  C=4.800000000000002, gamma=3.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=4.800000000000002, gamma=3.0, kernel=rbf ......................
[CV]  C=4.800000000000002, gamma=3.0, kernel=rbf, score=0.628, total=   0.0s
[CV] C=4.800000000000002, gamma=3.0, kernel=rbf ......................
[CV]  C=4.800000000000002, gamma=3.0, kernel=rbf, score=0.538, total=   0.0s
[CV] C=4.800000000000002, gamma=3.0, kernel=rbf ......................
[CV]  C=4.800000000000002, gamma=3.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=4.800000000000002, gamma=4.0, kernel=rbf ......................
[CV]  C=4.800000000000002, gamma=4.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=4.800000000000002, gamma=4.0, kernel=rbf ......................
[CV]  C=4.800000000000002, gamma=4.0, kernel=rbf, score=0.590, total=   0.0s
[CV] C=4.800000000000002, gamma=4.0, kernel=rbf ......................
[CV]  C=4.800000000000002, gamma=4.0, ker

[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   19.1s finished


In [21]:
# 최고의 결과가 나온 경우의 파라미터 값
print(model_svm.best_params_)

{'C': 3.0, 'gamma': 1.0, 'kernel': 'rbf'}


In [22]:
# 이를 적용해서 찍은 accuracy

svm_temp = SVC(kernel='rbf',gamma = 1.0, C = 3.0)
svm_temp.fit(X_u_s,y_under)
y_pred = svm_temp.predict(X_test)

print('Accuracy Score:')
print(metrics.accuracy_score(y_test,y_pred))

Accuracy Score:
0.0017906674625188723


이 데이터는 kernel을 linear한 것을 사용해야함을 알 수 있다.  
rbf 보다 linear 사용했을 때의 성능이 월등히 높음